# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7ff188521fa0>
├── 'a' --> tensor([[-1.0185, -1.1625,  1.4033],
│                   [-0.6400, -1.3386,  0.6276]])
└── 'x' --> <FastTreeValue 0x7ff188521ee0>
    └── 'c' --> tensor([[-0.7607,  2.4333, -0.4083, -0.6661],
                        [ 0.1746,  0.6261,  0.1945, -2.8896],
                        [ 0.5003, -1.0422,  0.6965,  0.7633]])

In [4]:
t.a

tensor([[-1.0185, -1.1625,  1.4033],
        [-0.6400, -1.3386,  0.6276]])

In [5]:
%timeit t.a

64.5 ns ± 0.0978 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7ff188521fa0>
├── 'a' --> tensor([[-0.7119, -1.5440, -0.2076],
│                   [ 0.6398,  0.3553, -1.0343]])
└── 'x' --> <FastTreeValue 0x7ff188521ee0>
    └── 'c' --> tensor([[-0.7607,  2.4333, -0.4083, -0.6661],
                        [ 0.1746,  0.6261,  0.1945, -2.8896],
                        [ 0.5003, -1.0422,  0.6965,  0.7633]])

In [7]:
%timeit t.a = new_value

67.7 ns ± 0.043 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.0185, -1.1625,  1.4033],
               [-0.6400, -1.3386,  0.6276]]),
    x: Batch(
           c: tensor([[-0.7607,  2.4333, -0.4083, -0.6661],
                      [ 0.1746,  0.6261,  0.1945, -2.8896],
                      [ 0.5003, -1.0422,  0.6965,  0.7633]]),
       ),
)

In [10]:
b.a

tensor([[-1.0185, -1.1625,  1.4033],
        [-0.6400, -1.3386,  0.6276]])

In [11]:
%timeit b.a

58.7 ns ± 0.0767 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.0111,  0.1618, -0.8218],
               [-0.7485, -0.7592,  0.2147]]),
    x: Batch(
           c: tensor([[-0.7607,  2.4333, -0.4083, -0.6661],
                      [ 0.1746,  0.6261,  0.1945, -2.8896],
                      [ 0.5003, -1.0422,  0.6965,  0.7633]]),
       ),
)

In [13]:
%timeit b.a = new_value

498 ns ± 0.206 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

835 ns ± 0.0992 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.9 µs ± 38.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

143 µs ± 315 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

143 µs ± 800 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7ff0db187dc0>
├── 'a' --> tensor([[[-1.0185, -1.1625,  1.4033],
│                    [-0.6400, -1.3386,  0.6276]],
│           
│                   [[-1.0185, -1.1625,  1.4033],
│                    [-0.6400, -1.3386,  0.6276]],
│           
│                   [[-1.0185, -1.1625,  1.4033],
│                    [-0.6400, -1.3386,  0.6276]],
│           
│                   [[-1.0185, -1.1625,  1.4033],
│                    [-0.6400, -1.3386,  0.6276]],
│           
│                   [[-1.0185, -1.1625,  1.4033],
│                    [-0.6400, -1.3386,  0.6276]],
│           
│                   [[-1.0185, -1.1625,  1.4033],
│                    [-0.6400, -1.3386,  0.6276]],
│           
│                   [[-1.0185, -1.1625,  1.4033],
│                    [-0.6400, -1.3386,  0.6276]],
│           
│                   [[-1.0185, -1.1625,  1.4033],
│                    [-0.6400, -1.3386,  0.6276]]])
└── 'x' --> <FastTreeValue 0x7ff0db1872b0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.2 µs ± 140 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7ff188518460>
├── 'a' --> tensor([[-1.0185, -1.1625,  1.4033],
│                   [-0.6400, -1.3386,  0.6276],
│                   [-1.0185, -1.1625,  1.4033],
│                   [-0.6400, -1.3386,  0.6276],
│                   [-1.0185, -1.1625,  1.4033],
│                   [-0.6400, -1.3386,  0.6276],
│                   [-1.0185, -1.1625,  1.4033],
│                   [-0.6400, -1.3386,  0.6276],
│                   [-1.0185, -1.1625,  1.4033],
│                   [-0.6400, -1.3386,  0.6276],
│                   [-1.0185, -1.1625,  1.4033],
│                   [-0.6400, -1.3386,  0.6276],
│                   [-1.0185, -1.1625,  1.4033],
│                   [-0.6400, -1.3386,  0.6276],
│                   [-1.0185, -1.1625,  1.4033],
│                   [-0.6400, -1.3386,  0.6276]])
└── 'x' --> <FastTreeValue 0x7ff15c971340>
    └── 'c' --> tensor([[-0.7607,  2.4333, -0.4083, -0.6661],
                        [ 0.1746,  0.6261,  0.1945, -2.8896],
                 

In [23]:
%timeit t_cat(trees)

30.3 µs ± 43.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

55.8 µs ± 15 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.7607,  2.4333, -0.4083, -0.6661],
                       [ 0.1746,  0.6261,  0.1945, -2.8896],
                       [ 0.5003, -1.0422,  0.6965,  0.7633]],
              
                      [[-0.7607,  2.4333, -0.4083, -0.6661],
                       [ 0.1746,  0.6261,  0.1945, -2.8896],
                       [ 0.5003, -1.0422,  0.6965,  0.7633]],
              
                      [[-0.7607,  2.4333, -0.4083, -0.6661],
                       [ 0.1746,  0.6261,  0.1945, -2.8896],
                       [ 0.5003, -1.0422,  0.6965,  0.7633]],
              
                      [[-0.7607,  2.4333, -0.4083, -0.6661],
                       [ 0.1746,  0.6261,  0.1945, -2.8896],
                       [ 0.5003, -1.0422,  0.6965,  0.7633]],
              
                      [[-0.7607,  2.4333, -0.4083, -0.6661],
                       [ 0.1746,  0.6261,  0.1945, -2.8896],
                       [ 0.5003, -1.0422,  0.6965,  0.7633]],

In [26]:
%timeit Batch.stack(batches)

77.5 µs ± 65.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.7607,  2.4333, -0.4083, -0.6661],
                      [ 0.1746,  0.6261,  0.1945, -2.8896],
                      [ 0.5003, -1.0422,  0.6965,  0.7633],
                      [-0.7607,  2.4333, -0.4083, -0.6661],
                      [ 0.1746,  0.6261,  0.1945, -2.8896],
                      [ 0.5003, -1.0422,  0.6965,  0.7633],
                      [-0.7607,  2.4333, -0.4083, -0.6661],
                      [ 0.1746,  0.6261,  0.1945, -2.8896],
                      [ 0.5003, -1.0422,  0.6965,  0.7633],
                      [-0.7607,  2.4333, -0.4083, -0.6661],
                      [ 0.1746,  0.6261,  0.1945, -2.8896],
                      [ 0.5003, -1.0422,  0.6965,  0.7633],
                      [-0.7607,  2.4333, -0.4083, -0.6661],
                      [ 0.1746,  0.6261,  0.1945, -2.8896],
                      [ 0.5003, -1.0422,  0.6965,  0.7633],
                      [-0.7607,  2.4333, -0.4083, -0.6661],
                   

In [28]:
%timeit Batch.cat(batches)

144 µs ± 178 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

319 µs ± 1.49 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
